# 模組 2: 企業級元資料管理與索引系統
## Module 2: Enterprise Metadata Management & Indexing Systems

> **教學目標**: 設計企業級元資料 schema、建立多層索引系統、實作混合檢索策略

---

## 📚 學習成果 (Learning Outcomes)

完成本模組後，您將能夠：
1. **設計可擴展的元資料 schema** - 支援多文檔類型的統一模型
2. **建立企業級索引系統** - 向量索引、關鍵字索引、混合索引
3. **實作高效檢索策略** - 語義檢索 + 關鍵字檢索的融合
4. **建立元資料治理機制** - 版本控制、血緣追蹤、品質監控

---

## 🎯 核心概念 (Core Concepts)

### 1. 企業元資料架構
- **統一資料模型**: 支援論文、報告、政策等多類型文檔
- **元資料血緣**: 追蹤資料來源與處理歷程
- **語義標註**: 自動提取實體、關係、概念
- **品質指標**: 完整性、準確性、一致性評估

### 2. 多層索引策略
- **向量索引**: 高維語義嵌入檢索
- **倒排索引**: 傳統關鍵字檢索
- **圖索引**: 實體關係網絡檢索
- **混合索引**: 融合多種檢索方式

### 3. 檢索增強技術
- **查詢重寫**: 擴展使用者查詢意圖
- **結果重排**: 基於相關性和多樣性
- **個人化**: 根據使用者歷史調整結果
- **解釋性**: 提供檢索理由說明

## 🔧 環境設定與依賴安裝

In [ ]:
# 安裝核心依賴套件
!pip install chromadb sentence-transformers sqlalchemy alembic
!pip install whoosh elasticsearch-dsl rank-bm25
!pip install networkx plotly pandas numpy scikit-learn
!pip install pydantic fastapi uvicorn

# 安裝 NLP 相關套件
!pip install spacy transformers torch
!python -m spacy download en_core_web_sm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# 導入必要的函式庫
import os
import sys
import json
import sqlite3
import hashlib
import asyncio
from pathlib import Path
from datetime import datetime, timezone
from typing import List, Dict, Optional, Tuple, Any, Union
from dataclasses import dataclass, field, asdict
from enum import Enum
import warnings
warnings.filterwarnings('ignore')

# 資料處理與分析
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# 嵌入與向量處理
import chromadb
from sentence_transformers import SentenceTransformer
import torch

# 資料庫與 ORM
from sqlalchemy import create_engine, Column, String, Integer, Float, DateTime, Text, JSON, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.dialects.sqlite import insert

# NLP 處理
import spacy
from rank_bm25 import BM25Okapi

# 資料驗證
from pydantic import BaseModel, Field, validator

# 可視化
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx

# 設定專案路徑
PROJECT_ROOT = Path('/home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance')
sys.path.append(str(PROJECT_ROOT))

print("✅ 環境設定完成")
print(f"📂 專案根目錄: {PROJECT_ROOT}")
print(f"🤖 CUDA 可用: {torch.cuda.is_available()}")

✅ 環境設定完成
📂 專案根目錄: /home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance
🤖 CUDA 可用: True


## 📋 企業級元資料模型設計

### 統一元資料 Schema

In [ ]:
# SQLAlchemy 基礎設定
from datetime import datetime
from typing import Any


Base = declarative_base()

class DocumentType(Enum):
    """文檔類型枚舉"""
    ACADEMIC_PAPER = "academic_paper"
    BUSINESS_REPORT = "business_report"
    TECHNICAL_DOC = "technical_doc"
    POLICY = "policy"
    PROCEDURE = "procedure"
    OTHER = "other"

class ProcessingStatus(Enum):
    """處理狀態枚舉"""
    PENDING = "pending"
    PROCESSING = "processing"
    COMPLETED = "completed"
    FAILED = "failed"
    ARCHIVED = "archived"

class SecurityLevel(Enum):
    """安全等級枚舉"""
    PUBLIC = "public"
    INTERNAL = "internal"
    CONFIDENTIAL = "confidential"
    RESTRICTED = "restricted"

# 企業級文檔元資料表
class EnterpriseDocument(Base):
    """企業級文檔元資料表"""
    __tablename__ = 'documents'
    
    # 基礎識別資訊
    document_id = Column(String(50), primary_key=True)
    title = Column(String(500), nullable=False, index=True)
    subtitle = Column(String(500))
    
    # 內容分類
    document_type = Column(String(50), nullable=False, index=True)
    category = Column(String(100), index=True)
    subcategory = Column(String(100))
    
    # 作者與來源
    authors = Column(JSON)  # List of author names
    organization = Column(String(200))
    department = Column(String(100))
    source_url = Column(String(1000))
    
    # 檔案資訊
    file_path = Column(String(1000), nullable=False)
    file_name = Column(String(255), nullable=False)
    file_size = Column(Integer)
    file_format = Column(String(10))
    
    # 內容統計
    page_count = Column(Integer, default=0)
    word_count = Column(Integer, default=0)
    paragraph_count = Column(Integer, default=0)
    
    # 語言與地區
    language = Column(String(10), default='en')
    region = Column(String(50))
    
    # 時間戳記
    created_date = Column(DateTime, nullable=False)
    modified_date = Column(DateTime)
    processed_date = Column(DateTime)
    ingested_date = Column(DateTime, default=datetime.now)
    
    # 處理狀態
    processing_status = Column(String(20), default='pending', index=True)
    
    # 品質與安全
    quality_score = Column(Float, default=0.0, index=True)
    security_level = Column(String(20), default='internal')
    
    # 語義標籤
    keywords = Column(JSON)  # List of extracted keywords
    entities = Column(JSON)  # Named entities
    topics = Column[Any](JSON)    # Topic modeling results
    
    # 版本控制
    version = Column(String(20), default='1.0')
    parent_id = Column(String(50), ForeignKey('documents.document_id'))
    
    # 使用統計
    access_count = Column(Integer, default=0)
    last_accessed = Column(DateTime)
    
    # 自訂欄位
    custom_metadata = Column(JSON)  # Flexible additional metadata
    
    # 關聯
    chunks = relationship("DocumentChunk", back_populates="document", cascade="all, delete-orphan")
    lineage_records = relationship("DataLineage", back_populates="document")

# 文檔分塊表
class DocumentChunk(Base):
    """文檔分塊表"""
    __tablename__ = 'document_chunks'
    
    chunk_id = Column(String(50), primary_key=True)
    document_id = Column(String(50), ForeignKey('documents.document_id'), nullable=False, index=True)
    
    # 內容資訊
    content = Column(Text, nullable=False)
    content_hash = Column(String(64), index=True)  # SHA-256 of content
    
    # 位置資訊
    chunk_index = Column(Integer, nullable=False)
    start_page = Column(Integer)
    end_page = Column(Integer)
    start_offset = Column(Integer)
    end_offset = Column(Integer)
    
    # 統計資訊
    word_count = Column(Integer, nullable=False)
    sentence_count = Column(Integer)
    
    # 語義類型
    semantic_type = Column(String(50), default='text')  # text, table, figure, header, etc.
    
    # 向量嵌入
    embedding_model = Column(String(100))
    embedding_dim = Column(Integer)
    
    # 品質指標
    chunk_quality = Column(Float, default=0.0)
    
    # 時間戳記
    created_date = Column(DateTime, default=datetime.now)
    
    # 關聯
    document = relationship("EnterpriseDocument", back_populates="chunks")

# 資料血緣追蹤表
class DataLineage(Base):
    """資料血緣追蹤表"""
    __tablename__ = 'data_lineage'
    
    lineage_id = Column[str](String(50), primary_key=True)
    document_id = Column[str](String(50), ForeignKey('documents.document_id'), nullable=False)
    
    # 處理階段
    stage = Column[str](String(50), nullable=False)  # ingestion, processing, indexing, etc.
    operation = Column[str](String(100), nullable=False)  # extract, chunk, embed, etc.
    
    # 輸入輸出
    input_data = Column[Any](JSON)   # Input parameters/data
    output_data = Column[Any](JSON)  # Output results
    
    # 處理資訊
    processor = Column[str](String(100))  # Which component processed
    processor_version = Column[str](String(20))
    processing_time_ms = Column[int](Integer)
    
    # 品質指標
    success = Column[int](Integer, default=1)  # 1=success, 0=failure
    error_message = Column[str](Text)
    
    # 時間戳記
    timestamp = Column[datetime](DateTime, default=datetime.now, index=True)
    
    # 關聯
    document = relationship("EnterpriseDocument", back_populates="lineage_records")

print("✅ 企業級元資料模型定義完成")

✅ 企業級元資料模型定義完成


### Pydantic 資料驗證模型

In [ ]:
# Pydantic 模型用於資料驗證和 API 介面

class DocumentMetadataRequest(BaseModel):
    """文檔元資料請求模型"""
    title: str = Field(..., min_length=1, max_length=500)
    subtitle: Optional[str] = Field(None, max_length=500)
    document_type: str = Field(..., pattern="^(academic_paper|business_report|technical_doc|policy|procedure|other)$")
    category: Optional[str] = Field(None, max_length=100)
    authors: List[str] = Field(default_factory=list)
    organization: Optional[str] = Field(None, max_length=200)
    language: str = Field(default="en", pattern="^[a-z]{2}$")
    security_level: str = Field(default="internal", pattern="^(public|internal|confidential|restricted)$")
    keywords: List[str] = Field(default_factory=list)
    custom_metadata: Dict[str, Any] = Field(default_factory=dict)
    
    @validator('keywords')
    def validate_keywords(cls, v):
        if len(v) > 50:  # 限制關鍵字數量
            raise ValueError('Too many keywords (max 50)')
        return v

class DocumentChunkRequest(BaseModel):
    """文檔分塊請求模型"""
    content: str = Field(..., min_length=1)
    chunk_index: int = Field(..., ge=0)
    semantic_type: str = Field(default="text")
    start_page: Optional[int] = Field(None, ge=1)
    end_page: Optional[int] = Field(None, ge=1)
    
    @validator('content')
    def validate_content_length(cls, v):
        if len(v.split()) < 5:  # 至少5個詞
            raise ValueError('Content too short (min 5 words)')
        return v

class SearchQuery(BaseModel):
    """搜尋查詢模型"""
    query: str = Field(..., min_length=1, max_length=1000)
    search_type: str = Field(default="hybrid", pattern="^(semantic|keyword|hybrid)$")
    document_types: List[str] = Field(default_factory=list)
    categories: List[str] = Field(default_factory=list)
    date_from: Optional[datetime] = None
    date_to: Optional[datetime] = None
    min_quality_score: float = Field(default=0.0, ge=0.0, le=1.0)
    limit: int = Field(default=10, ge=1, le=100)
    include_chunks: bool = Field(default=False)

class SearchResult(BaseModel):
    """搜尋結果模型"""
    document_id: str
    title: str
    document_type: str
    relevance_score: float
    snippet: str
    matched_chunks: List[str] = Field(default_factory=list)
    metadata: Dict[str, Any] = Field(default_factory=dict)

print("✅ Pydantic 驗證模型定義完成")

✅ Pydantic 驗證模型定義完成


## 🔍 企業級元資料管理器

### 核心管理器實作

In [ ]:
from sqlalchemy.orm.session import Session


class EnterpriseMetadataManager:
    """
    企業級元資料管理器
    統一管理文檔元資料的 CRUD 操作
    """
    
    def __init__(self, db_path: str, create_tables: bool = True):
        """初始化元資料管理器"""
        self.db_path = db_path
        self.engine = create_engine(f"sqlite:///{db_path}", echo=False)
        self.SessionLocal = sessionmaker[Session](autocommit=False, autoflush=False, bind=self.engine)
        
        if create_tables:
            Base.metadata.create_all(bind=self.engine)
            print(f"✅ 資料庫表已建立: {db_path}")
        
        # 初始化 NLP 模型
        self.nlp = spacy.load('en_core_web_sm')
        print("✅ NLP 模型載入完成")
    
    def get_session(self):
        """獲取資料庫會話"""
        return self.SessionLocal()
    
    def create_document(self, file_path: str, content: str, metadata_request: DocumentMetadataRequest) -> str:
        """
        建立新文檔記錄
        
        Args:
            file_path: 檔案路徑
            content: 檔案內容
            metadata_request: 元資料請求
            
        Returns:
            str: 文檔 ID
        """
        session = self.get_session()
        
        try:
            # 生成文檔 ID
            document_id = self._generate_document_id(file_path, metadata_request.title)
            
            # 提取檔案資訊
            file_info = self._extract_file_info(file_path)
            
            # 分析內容統計
            content_stats = self._analyze_content(content)
            
            # 提取語義資訊
            semantic_info = self._extract_semantic_info(content)
            
            # 建立文檔記錄
            document = EnterpriseDocument(
                document_id=document_id,
                title=metadata_request.title,
                subtitle=metadata_request.subtitle,
                document_type=metadata_request.document_type,
                category=metadata_request.category,
                authors=metadata_request.authors,
                organization=metadata_request.organization,
                file_path=str(file_path),
                file_name=file_info['name'],
                file_size=file_info['size'],
                file_format=file_info['format'],
                word_count=content_stats['word_count'],
                paragraph_count=content_stats['paragraph_count'],
                language=metadata_request.language,
                created_date=file_info['created_date'],
                modified_date=file_info['modified_date'],
                processed_date=datetime.now(),
                processing_status='completed',
                security_level=metadata_request.security_level,
                keywords=metadata_request.keywords + semantic_info['keywords'],
                entities=semantic_info['entities'],
                topics=semantic_info['topics'],
                custom_metadata=metadata_request.custom_metadata
            )
            
            session.add(document)
            session.commit()
            
            # 記錄血緣 - 修正Pydantic模型轉換
            self._record_lineage(session, document_id, 'metadata_creation', 'create_document', 
                               {'input': metadata_request.dict()}, {'document_id': document_id})
            
            print(f"✅ 文檔已建立: {document_id}")
            return document_id
            
        except Exception as e:
            session.rollback()
            print(f"❌ 建立文檔失敗: {e}")
            raise
        finally:
            session.close()
    
    def add_document_chunks(self, document_id: str, chunks_data: List[DocumentChunkRequest]) -> List[str]:
        """
        為文檔添加分塊
        
        Args:
            document_id: 文檔 ID
            chunks_data: 分塊資料列表
            
        Returns:
            List[str]: 分塊 ID 列表
        """
        session = self.get_session()
        
        try:
            chunk_ids = []
            
            for chunk_data in chunks_data:
                chunk_id = f"{document_id}_chunk_{chunk_data.chunk_index}"
                content_hash = hashlib.sha256(chunk_data.content.encode()).hexdigest()
                
                chunk = DocumentChunk(
                    chunk_id=chunk_id,
                    document_id=document_id,
                    content=chunk_data.content,
                    content_hash=content_hash,
                    chunk_index=chunk_data.chunk_index,
                    start_page=chunk_data.start_page,
                    end_page=chunk_data.end_page,
                    word_count=len(chunk_data.content.split()),
                    sentence_count=len(list(self.nlp(chunk_data.content).sents)),
                    semantic_type=chunk_data.semantic_type,
                )
                
                session.add(chunk)
                chunk_ids.append(chunk_id)
            
            session.commit()
            
            # 記錄血緣
            self._record_lineage(session, document_id, 'chunking', 'add_chunks',
                               {'chunk_count': len(chunks_data)}, {'chunk_ids': chunk_ids})
            
            print(f"✅ 已添加 {len(chunk_ids)} 個分塊")
            return chunk_ids
            
        except Exception as e:
            session.rollback()
            print(f"❌ 添加分塊失敗: {e}")
            raise
        finally:
            session.close()
    
    def get_document(self, document_id: str, include_chunks: bool = False) -> Optional[Dict]:
        """獲取文檔資訊"""
        session = self.get_session()
        
        try:
            # 查詢文檔
            document = session.query(EnterpriseDocument).filter(
                EnterpriseDocument.document_id == document_id
            ).first()
            
            if not document:
                return None
            
            # 轉換為字典
            doc_dict = {
                'document_id': document.document_id,
                'title': document.title,
                'subtitle': document.subtitle,
                'document_type': document.document_type,
                'category': document.category,
                'authors': document.authors or [],
                'organization': document.organization,
                'file_path': document.file_path,
                'file_name': document.file_name,
                'file_size': document.file_size,
                'word_count': document.word_count,
                'language': document.language,
                'created_date': document.created_date,
                'processed_date': document.processed_date,
                'quality_score': document.quality_score,
                'security_level': document.security_level,
                'keywords': document.keywords or [],
                'entities': document.entities or {},
                'topics': document.topics or [],
                'custom_metadata': document.custom_metadata or {}
            }
            
            # 包含分塊資訊
            if include_chunks:
                chunks = session.query(DocumentChunk).filter(
                    DocumentChunk.document_id == document_id
                ).order_by(DocumentChunk.chunk_index).all()
                
                doc_dict['chunks'] = [{
                    'chunk_id': chunk.chunk_id,
                    'content': chunk.content,
                    'chunk_index': chunk.chunk_index,
                    'word_count': chunk.word_count,
                    'semantic_type': chunk.semantic_type
                } for chunk in chunks]
            
            return doc_dict
            
        finally:
            session.close()
    
    def list_documents(self, document_type: Optional[str] = None, 
                      category: Optional[str] = None,
                      limit: int = 100) -> List[Dict]:
        """列出文檔"""
        session = self.get_session()
        
        try:
            query = session.query(EnterpriseDocument)
            
            if document_type:
                query = query.filter(EnterpriseDocument.document_type == document_type)
            
            if category:
                query = query.filter(EnterpriseDocument.category == category)
            
            documents = query.order_by(EnterpriseDocument.processed_date.desc()).limit(limit).all()
            
            return [{
                'document_id': doc.document_id,
                'title': doc.title,
                'document_type': doc.document_type,
                'category': doc.category,
                'authors': doc.authors or [],
                'word_count': doc.word_count,
                'quality_score': doc.quality_score,
                'processed_date': doc.processed_date
            } for doc in documents]
            
        finally:
            session.close()
    
    def update_quality_score(self, document_id: str, quality_score: float) -> bool:
        """更新文檔品質分數"""
        session = self.get_session()
        
        try:
            document = session.query(EnterpriseDocument).filter(
                EnterpriseDocument.document_id == document_id
            ).first()
            
            if document:
                old_score = document.quality_score
                document.quality_score = quality_score
                session.commit()
                
                # 記錄血緣
                self._record_lineage(session, document_id, 'quality_update', 'update_score',
                                   {'old_score': old_score}, {'new_score': quality_score})
                
                print(f"✅ 品質分數已更新: {old_score} → {quality_score}")
                return True
            
            return False
            
        except Exception as e:
            session.rollback()
            print(f"❌ 更新品質分數失敗: {e}")
            return False
        finally:
            session.close()

print("✅ EnterpriseMetadataManager 部分方法定義完成")

✅ EnterpriseMetadataManager 部分方法定義完成


### 輔助方法實作

In [ ]:
# 繼續 EnterpriseMetadataManager 的輔助方法

def _generate_document_id(self, file_path: str, title: str) -> str:
    """生成唯一文檔 ID"""
    timestamp = datetime.now().isoformat()
    content = f"{file_path}_{title}_{timestamp}"
    return hashlib.md5(content.encode()).hexdigest()[:16]

def _extract_file_info(self, file_path: str) -> Dict:
    """提取檔案基本資訊"""
    path = Path(file_path)
    
    if path.exists():
        stats = path.stat()
        return {
            'name': path.name,
            'size': stats.st_size,
            'format': path.suffix.lower().lstrip('.'),
            'created_date': datetime.fromtimestamp(stats.st_ctime),
            'modified_date': datetime.fromtimestamp(stats.st_mtime)
        }
    else:
        return {
            'name': path.name,
            'size': 0,
            'format': path.suffix.lower().lstrip('.'),
            'created_date': datetime.now(),
            'modified_date': datetime.now()
        }

def _analyze_content(self, content: str) -> Dict:
    """分析內容統計資訊"""
    if not content.strip():
        return {'word_count': 0, 'paragraph_count': 0}
    
    words = content.split()
    paragraphs = [p.strip() for p in content.split('\n\n') if p.strip()]
    
    return {
        'word_count': len(words),
        'paragraph_count': len(paragraphs)
    }

def _extract_semantic_info(self, content: str) -> Dict:
    """
    提取語義資訊
    包含實體、關鍵字、主題
    """
    if not content.strip():
        return {'keywords': [], 'entities': {}, 'topics': []}
    
    # 使用 spaCy 處理
    doc = self.nlp(content[:10000])  # 限制長度避免記憶體問題
    
    # 提取命名實體
    entities = {}
    for ent in doc.ents:
        ent_type = ent.label_
        if ent_type not in entities:
            entities[ent_type] = []
        if ent.text not in entities[ent_type]:
            entities[ent_type].append(ent.text)
    
    # 提取關鍵詞（名詞短語）
    keywords = []
    for chunk in doc.noun_chunks:
        if 2 <= len(chunk.text.split()) <= 4:  # 2-4詞的名詞短語
            keywords.append(chunk.text.lower())
    
    # 去重並限制數量
    keywords = list(set(keywords))[:20]
    
    # 簡化的主題提取（基於頻繁詞）
    important_words = [token.lemma_.lower() for token in doc 
                      if token.is_alpha and not token.is_stop and len(token.text) > 3]
    
    from collections import Counter
    word_freq = Counter(important_words)
    topics = [word for word, count in word_freq.most_common(10) if count >= 2]
    
    return {
        'keywords': keywords,
        'entities': entities,
        'topics': topics
    }

def _record_lineage(self, session, document_id: str, stage: str, operation: str, 
                   input_data: Dict, output_data: Dict) -> None:
    """記錄資料血緣"""
    lineage_id = f"{document_id}_{stage}_{operation}_{int(datetime.now().timestamp())}"
    
    lineage = DataLineage(
        lineage_id=lineage_id,
        document_id=document_id,
        stage=stage,
        operation=operation,
        input_data=input_data,
        output_data=output_data,
        processor='EnterpriseMetadataManager',
        processor_version='1.0',
        success=1
    )
    
    session.add(lineage)
    session.commit()

def get_lineage_history(self, document_id: str) -> List[Dict]:
    """獲取文檔的血緣歷史"""
    session = self.get_session()
    
    try:
        lineages = session.query(DataLineage).filter(
            DataLineage.document_id == document_id
        ).order_by(DataLineage.timestamp.asc()).all()
        
        return [{
            'lineage_id': lineage.lineage_id,
            'stage': lineage.stage,
            'operation': lineage.operation,
            'processor': lineage.processor,
            'timestamp': lineage.timestamp,
            'success': lineage.success,
            'input_data': lineage.input_data,
            'output_data': lineage.output_data
        } for lineage in lineages]
        
    finally:
        session.close()

def generate_statistics(self) -> Dict:
    """生成元資料統計報告"""
    session = self.get_session()
    
    try:
        # 基本統計
        total_docs = session.query(EnterpriseDocument).count()
        total_chunks = session.query(DocumentChunk).count()
        
        # 按類型統計
        type_stats = session.query(
            EnterpriseDocument.document_type,
            func.count(EnterpriseDocument.document_id).label('count')
        ).group_by(EnterpriseDocument.document_type).all()
        
        # 按狀態統計
        status_stats = session.query(
            EnterpriseDocument.processing_status,
            func.count(EnterpriseDocument.document_id).label('count')
        ).group_by(EnterpriseDocument.processing_status).all()
        
        # 品質分布
        quality_avg = session.query(
            func.avg(EnterpriseDocument.quality_score)
        ).scalar() or 0.0
        
        return {
            'total_documents': total_docs,
            'total_chunks': total_chunks,
            'avg_chunks_per_doc': total_chunks / total_docs if total_docs > 0 else 0,
            'document_types': {row.document_type: row.count for row in type_stats},
            'processing_status': {row.processing_status: row.count for row in status_stats},
            'average_quality_score': round(quality_avg, 3)
        }
        
    finally:
        session.close()

# 將方法添加到 EnterpriseMetadataManager 類別
from sqlalchemy import func

EnterpriseMetadataManager._generate_document_id = _generate_document_id
EnterpriseMetadataManager._extract_file_info = _extract_file_info
EnterpriseMetadataManager._analyze_content = _analyze_content
EnterpriseMetadataManager._extract_semantic_info = _extract_semantic_info
EnterpriseMetadataManager._record_lineage = _record_lineage
EnterpriseMetadataManager.get_lineage_history = get_lineage_history
EnterpriseMetadataManager.generate_statistics = generate_statistics

print("✅ EnterpriseMetadataManager 輔助方法定義完成")

✅ EnterpriseMetadataManager 輔助方法定義完成


## 🔍 多層索引系統實作

### 向量索引管理器

In [ ]:
class VectorIndexManager:
    """
    向量索引管理器
    使用 ChromaDB 管理向量嵌入和語義搜索
    """
    
    def __init__(self, 
                 chroma_path: str,
                 embedding_model: str = "all-MiniLM-L6-v2",
                 collection_name: str = "document_chunks"):
        """
        初始化向量索引管理器
        
        Args:
            chroma_path: ChromaDB 存儲路徑
            embedding_model: 嵌入模型名稱
            collection_name: 集合名稱
        """
        self.chroma_path = chroma_path
        self.embedding_model_name = embedding_model
        self.collection_name = collection_name
        
        # 初始化嵌入模型
        print(f"🔄 載入嵌入模型: {embedding_model}")
        self.embedding_model = SentenceTransformer(embedding_model)
        print(f"✅ 嵌入模型載入完成")
        
        # 初始化 ChromaDB
        self.client = chromadb.PersistentClient(path=chroma_path)
        
        # 建立或獲取集合
        try:
            self.collection = self.client.create_collection(
                name=collection_name,
                metadata={"hnsw:space": "cosine"},  # 使用餘弦相似度
            )
            print(f"✅ 建立新集合: {collection_name}")
        except Exception:
            self.collection = self.client.get_collection(name=collection_name)
            print(f"✅ 使用現有集合: {collection_name}")
        
        print(f"📊 集合統計: {self.collection.count()} 個向量")
    
    def add_document_embeddings(self, document_id: str, chunks: List[Dict]) -> List[str]:
        """
        為文檔分塊生成並存儲向量嵌入
        
        Args:
            document_id: 文檔 ID
            chunks: 分塊資料列表
            
        Returns:
            List[str]: 已處理的分塊 ID 列表
        """
        if not chunks:
            return []
        
        print(f"🔄 處理 {len(chunks)} 個分塊的向量嵌入...")
        
        # 準備資料
        chunk_ids = []
        documents = []
        metadatas = []
        
        for chunk in chunks:
            chunk_id = chunk.get('chunk_id') or f"{document_id}_chunk_{chunk['chunk_index']}"
            content = chunk['content']
            
            # 過濾太短的內容
            if len(content.split()) < 5:
                continue
            
            chunk_ids.append(chunk_id)
            documents.append(content)
            metadatas.append({
                'document_id': document_id,
                'chunk_index': chunk['chunk_index'],
                'word_count': chunk.get('word_count', len(content.split())),
                'semantic_type': chunk.get('semantic_type', 'text'),
                'embedding_model': self.embedding_model_name,
                'created_date': datetime.now().isoformat()
            })
        
        if not chunk_ids:
            print("⚠️  沒有有效的分塊可處理")
            return []
        
        try:
            # 批次生成嵌入向量
            embeddings = self.embedding_model.encode(
                documents, 
                batch_size=32,
                show_progress_bar=True
            )
            
            # 添加到 ChromaDB
            self.collection.add(
                ids=chunk_ids,
                documents=documents,
                embeddings=embeddings.tolist(),
                metadatas=metadatas
            )
            
            print(f"✅ 成功添加 {len(chunk_ids)} 個向量嵌入")
            return chunk_ids
            
        except Exception as e:
            print(f"❌ 添加向量嵌入失敗: {e}")
            return []
    
    def semantic_search(self, 
                       query: str, 
                       top_k: int = 10,
                       document_types: Optional[List[str]] = None,
                       min_similarity: float = 0.0) -> List[Dict]:
        """
        執行語義搜索
        
        Args:
            query: 搜索查詢
            top_k: 返回結果數量
            document_types: 文檔類型過濾
            min_similarity: 最小相似度閾值
            
        Returns:
            List[Dict]: 搜索結果
        """
        if not query.strip():
            return []
        
        try:
            # 生成查詢向量
            query_embedding = self.embedding_model.encode([query])
            
            # 構建過濾條件
            where_filter = {}
            if document_types:
                # 注意：需要從文檔類型映射到實際的 document_id 過濾
                # 這裡簡化處理，實際應該查詢元資料管理器獲取對應的 document_id 列表
                pass
            
            # 執行搜索
            results = self.collection.query(
                query_embeddings=query_embedding.tolist(),
                n_results=top_k,
                where=where_filter if where_filter else None,
                include=["documents", "metadatas", "distances"]
            )
            
            # 處理結果
            search_results = []
            
            for i in range(len(results['ids'][0])):
                chunk_id = results['ids'][0][i]
                document = results['documents'][0][i]
                metadata = results['metadatas'][0][i]
                distance = results['distances'][0][i]
                
                # 轉換距離為相似度分數 (cosine distance -> similarity)
                similarity = 1.0 - distance
                
                # 應用相似度閾值過濾
                if similarity >= min_similarity:
                    search_results.append({
                        'chunk_id': chunk_id,
                        'document_id': metadata['document_id'],
                        'content': document,
                        'similarity_score': round(similarity, 4),
                        'chunk_index': metadata.get('chunk_index', 0),
                        'word_count': metadata.get('word_count', 0),
                        'semantic_type': metadata.get('semantic_type', 'text')
                    })
            
            print(f"🔍 語義搜索找到 {len(search_results)} 個結果")
            return search_results
            
        except Exception as e:
            print(f"❌ 語義搜索失敗: {e}")
            return []
    
    def get_similar_chunks(self, chunk_id: str, top_k: int = 5) -> List[Dict]:
        """
        找到與指定分塊相似的其他分塊
        
        Args:
            chunk_id: 分塊 ID
            top_k: 返回結果數量
            
        Returns:
            List[Dict]: 相似分塊列表
        """
        try:
            # 獲取目標分塊的向量
            target_result = self.collection.get(
                ids=[chunk_id],
                include=["embeddings", "documents", "metadatas"]
            )
            
            if not target_result['ids']:
                return []
            
            target_embedding = target_result['embeddings'][0]
            
            # 搜索相似向量
            results = self.collection.query(
                query_embeddings=[target_embedding],
                n_results=top_k + 1,  # +1 因為會包含自己
                include=["documents", "metadatas", "distances"]
            )
            
            # 過濾掉自己，返回其他相似分塊
            similar_chunks = []
            for i, returned_id in enumerate(results['ids'][0]):
                if returned_id != chunk_id:
                    similarity = 1.0 - results['distances'][0][i]
                    similar_chunks.append({
                        'chunk_id': returned_id,
                        'document_id': results['metadatas'][0][i]['document_id'],
                        'content': results['documents'][0][i],
                        'similarity_score': round(similarity, 4)
                    })
            
            return similar_chunks[:top_k]
            
        except Exception as e:
            print(f"❌ 獲取相似分塊失敗: {e}")
            return []
    
    def delete_document_embeddings(self, document_id: str) -> bool:
        """刪除文檔的所有向量嵌入"""
        try:
            # 查找文檔的所有分塊
            results = self.collection.get(
                where={"document_id": document_id},
                include=["metadatas"]
            )
            
            if results['ids']:
                self.collection.delete(ids=results['ids'])
                print(f"✅ 已刪除文檔 {document_id} 的 {len(results['ids'])} 個向量")
                return True
            else:
                print(f"⚠️  文檔 {document_id} 沒有向量嵌入")
                return False
                
        except Exception as e:
            print(f"❌ 刪除向量嵌入失敗: {e}")
            return False
    
    def get_collection_stats(self) -> Dict:
        """獲取集合統計資訊"""
        try:
            total_count = self.collection.count()
            
            # 獲取一些樣本來分析
            sample_size = min(100, total_count)
            if sample_size > 0:
                sample = self.collection.get(
                    limit=sample_size,
                    include=["metadatas"]
                )
                
                # 統計文檔分布
                doc_counts = {}
                semantic_types = {}
                
                for metadata in sample['metadatas']:
                    doc_id = metadata['document_id']
                    doc_counts[doc_id] = doc_counts.get(doc_id, 0) + 1
                    
                    sem_type = metadata.get('semantic_type', 'text')
                    semantic_types[sem_type] = semantic_types.get(sem_type, 0) + 1
                
                return {
                    'total_vectors': total_count,
                    'embedding_model': self.embedding_model_name,
                    'collection_name': self.collection_name,
                    'unique_documents': len(doc_counts),
                    'avg_chunks_per_doc': np.mean(list(doc_counts.values())),
                    'semantic_type_distribution': semantic_types
                }
            else:
                return {
                    'total_vectors': 0,
                    'embedding_model': self.embedding_model_name,
                    'collection_name': self.collection_name
                }
                
        except Exception as e:
            print(f"❌ 獲取統計資訊失敗: {e}")
            return {}

print("✅ VectorIndexManager 定義完成")

✅ VectorIndexManager 定義完成


## 🧪 實作練習: 企業元資料系統測試

### 建立測試環境

In [ ]:
# 設定測試環境路徑
test_dir = PROJECT_ROOT / 'notebooks' / '02_metadata_management' / 'test_data'
test_dir.mkdir(parents=True, exist_ok=True)

db_path = test_dir / 'enterprise_metadata.db'
chroma_path = test_dir / 'vector_index'

print(f"📂 測試目錄: {test_dir}")
print(f"🗄️  資料庫路徑: {db_path}")
print(f"🔍 向量索引路徑: {chroma_path}")

# 建立測試文檔
test_documents = {
    'transformer_paper.txt': """
Attention Is All You Need

Abstract
The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.

1. Introduction
Recurrent neural networks, long short-term memory and gated recurrent units in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation.

The goal of reducing sequential computation also leads to the foundation of the Extended Neural GPU, ByteNet and ConvS2S, all of which use convolutional neural networks as basic building block, computing hidden representations in parallel for all input and output positions.

In this work, we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output.

2. Background
The goal of reducing sequential computation also motivated the creation of the Extended Neural GPU, ByteNet and ConvS2S, all of which use convolutional neural networks as basic building blocks.

3. Model Architecture
Most competitive neural sequence transduction models have an encoder-decoder structure. Here, the encoder maps an input sequence to a sequence of continuous representations, which is then mapped to an output sequence by the decoder.

4. Why Self-Attention
In this section we compare various aspects of self-attention layers to the recurrent and convolutional layers commonly used for mapping one variable-length sequence of symbol representations to another sequence.
""",
    
    'bert_paper.txt': """
BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding

Abstract
We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers.

1. Introduction
Language model pre-training has been shown to be effective for improving many natural language processing tasks. These include sentence-level tasks such as natural language inference and paraphrasing, which aim to predict the relationships between sentences by analyzing them holistically.

There are two existing strategies for applying pre-trained language representations to downstream tasks: feature-based and fine-tuning. The feature-based approach uses task-specific architectures that include the pre-trained representations as additional features.

2. Related Work
There has been a long history of pre-training general language representations, and we briefly review the most widely-used approaches in this section.

2.1 Unsupervised Feature-based Approaches
Learning widely applicable representations of words has been an active area of research for decades, including non-neural and neural methods.

2.2 Unsupervised Fine-tuning Approaches
As with the feature-based approaches, the first works in this direction only pre-trained word embedding parameters from unlabeled text.

3. BERT
We introduce BERT and its detailed implementation in this section. There are two steps in our framework: pre-training and fine-tuning.
""",
    
    'ai_report.txt': """
Artificial Intelligence Market Analysis Report 2024

Executive Summary
The artificial intelligence market has experienced unprecedented growth in 2024, with global revenue reaching $180 billion, representing a 32% increase from the previous year.

Market Overview
The AI market is characterized by rapid technological advancement and increasing enterprise adoption across various industries. Machine learning and deep learning technologies continue to drive innovation.

Key Trends
1. Large Language Models (LLMs) have dominated the market with breakthrough applications in natural language processing.
2. Computer vision technologies have seen significant improvements in accuracy and efficiency.
3. Edge AI deployment has increased by 45% as organizations seek to reduce latency and improve privacy.

Industry Applications
Healthcare: AI-powered diagnostic tools have shown 95% accuracy in medical image analysis.
Finance: Algorithmic trading and fraud detection systems have reduced financial losses by 28%.
Automotive: Autonomous vehicle technology has advanced with Level 3 automation becoming commercially viable.

Market Projections
We project the AI market to reach $300 billion by 2026, driven by enterprise AI adoption and continued research breakthroughs.

Recommendations
1. Invest in AI talent development and training programs.
2. Develop comprehensive AI governance frameworks.
3. Focus on explainable AI to build trust and compliance.
"""
}

# 建立測試檔案
for filename, content in test_documents.items():
    file_path = test_dir / filename
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content)

print(f"✅ 建立了 {len(test_documents)} 個測試文檔")

📂 測試目錄: /home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance/notebooks/02_metadata_management/test_data
🗄️  資料庫路徑: /home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance/notebooks/02_metadata_management/test_data/enterprise_metadata.db
🔍 向量索引路徑: /home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance/notebooks/02_metadata_management/test_data/vector_index
✅ 建立了 3 個測試文檔


### 初始化管理器

In [ ]:
# 初始化元資料管理器
metadata_manager = EnterpriseMetadataManager(str(db_path), create_tables=True)

# 初始化向量索引管理器
vector_manager = VectorIndexManager(str(chroma_path))

print("✅ 管理器初始化完成")

✅ 資料庫表已建立: /home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance/notebooks/02_metadata_management/test_data/enterprise_metadata.db
✅ NLP 模型載入完成
🔄 載入嵌入模型: all-MiniLM-L6-v2
✅ 嵌入模型載入完成
✅ 使用現有集合: document_chunks
📊 集合統計: 0 個向量
✅ 管理器初始化完成


### 測試文檔處理流程

In [ ]:
# 處理測試文檔
print("🔄 開始處理測試文檔\n")

# 定義文檔元資料
documents_metadata = [
    {
        'file': 'transformer_paper.txt',
        'metadata': DocumentMetadataRequest(
            title="Attention Is All You Need",
            document_type="academic_paper",
            category="deep_learning",
            authors=["Ashish Vaswani", "Noam Shazeer", "Niki Parmar"],
            organization="Google Research",
            keywords=["transformer", "attention mechanism", "neural networks"],
            custom_metadata={
                "venue": "NIPS 2017",
                "citation_count": 50000,
                "field": "Natural Language Processing"
            }
        )
    },
    {
        'file': 'bert_paper.txt',
        'metadata': DocumentMetadataRequest(
            title="BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding",
            document_type="academic_paper",
            category="deep_learning",
            authors=["Jacob Devlin", "Ming-Wei Chang", "Kenton Lee"],
            organization="Google AI Language",
            keywords=["BERT", "bidirectional", "transformers", "pre-training"],
            custom_metadata={
                "venue": "NAACL 2019",
                "citation_count": 40000,
                "field": "Natural Language Processing"
            }
        )
    },
    {
        'file': 'ai_report.txt',
        'metadata': DocumentMetadataRequest(
            title="Artificial Intelligence Market Analysis Report 2024",
            document_type="business_report",
            category="market_analysis",
            authors=["AI Research Team"],
            organization="Tech Analytics Corp",
            keywords=["AI market", "machine learning", "industry trends"],
            custom_metadata={
                "report_type": "market_analysis",
                "year": 2024,
                "industry": "Technology"
            }
        )
    }
]

processed_documents = []

for doc_info in documents_metadata:
    file_path = test_dir / doc_info['file']
    
    # 讀取文檔內容
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 建立文檔記錄
    document_id = metadata_manager.create_document(
        str(file_path), content, doc_info['metadata']
    )
    
    # 簡單分塊處理（按段落）
    paragraphs = [p.strip() for p in content.split('\n\n') if p.strip()]
    
    chunks_data = []
    for i, paragraph in enumerate(paragraphs):
        if len(paragraph.split()) >= 10:  # 過濾太短的段落
            chunks_data.append(DocumentChunkRequest(
                content=paragraph,
                chunk_index=i,
                semantic_type="text"
            ))
    
    # 添加分塊
    chunk_ids = metadata_manager.add_document_chunks(document_id, chunks_data)
    
    # 為向量索引準備分塊資料
    chunk_dicts = [{
        'chunk_id': chunk_ids[i],
        'content': chunk.content,
        'chunk_index': chunk.chunk_index,
        'semantic_type': chunk.semantic_type
    } for i, chunk in enumerate(chunks_data)]
    
    # 添加向量嵌入
    vector_chunk_ids = vector_manager.add_document_embeddings(document_id, chunk_dicts)
    
    processed_documents.append({
        'document_id': document_id,
        'file': doc_info['file'],
        'chunk_count': len(chunk_ids),
        'vector_count': len(vector_chunk_ids)
    })
    
    print(f"✅ 已處理: {doc_info['file']} (文檔ID: {document_id}, 分塊: {len(chunk_ids)})\n")

print(f"🎉 所有文檔處理完成！")
for doc in processed_documents:
    print(f"📄 {doc['file']}: {doc['chunk_count']} 個分塊, {doc['vector_count']} 個向量")

🔄 開始處理測試文檔

✅ 文檔已建立: 5e2046078b2618a9
✅ 已添加 7 個分塊
🔄 處理 7 個分塊的向量嵌入...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 成功添加 7 個向量嵌入
✅ 已處理: transformer_paper.txt (文檔ID: 5e2046078b2618a9, 分塊: 7)

✅ 文檔已建立: f088c9e9011861a7
✅ 已添加 7 個分塊
🔄 處理 7 個分塊的向量嵌入...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 成功添加 7 個向量嵌入
✅ 已處理: bert_paper.txt (文檔ID: f088c9e9011861a7, 分塊: 7)

✅ 文檔已建立: 19b7b90a4bb6d87c
✅ 已添加 6 個分塊
🔄 處理 6 個分塊的向量嵌入...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 成功添加 6 個向量嵌入
✅ 已處理: ai_report.txt (文檔ID: 19b7b90a4bb6d87c, 分塊: 6)

🎉 所有文檔處理完成！
📄 transformer_paper.txt: 7 個分塊, 7 個向量
📄 bert_paper.txt: 7 個分塊, 7 個向量
📄 ai_report.txt: 6 個分塊, 6 個向量


### 測試搜索功能

In [ ]:
print("🔍 測試搜索功能\n")

# 測試語義搜索
test_queries = [
    "attention mechanism in neural networks",
    "transformer architecture and self-attention",
    "BERT bidirectional language model",
    "artificial intelligence market trends",
    "machine learning applications in healthcare"
]

for query in test_queries:
    print(f"🔎 查詢: '{query}'")
    
    # 執行語義搜索
    results = vector_manager.semantic_search(
        query=query,
        top_k=3,
        min_similarity=0.2
    )
    
    if results:
        print(f"找到 {len(results)} 個相關結果:\n")
        
        for i, result in enumerate(results, 1):
            print(f"  {i}. 相似度: {result['similarity_score']:.3f}")
            print(f"     文檔: {result['document_id']}")
            print(f"     分塊: {result['chunk_id']}")
            print(f"     內容: {result['content'][:100]}...")
            print()
    else:
        print("❌ 沒有找到相關結果\n")
    
    print("-" * 50)

🔍 測試搜索功能

🔎 查詢: 'attention mechanism in neural networks'
🔍 語義搜索找到 3 個結果
找到 3 個相關結果:

  1. 相似度: 0.568
     文檔: 5e2046078b2618a9
     分塊: 5e2046078b2618a9_chunk_7
     內容: 4. Why Self-Attention
In this section we compare various aspects of self-attention layers to the rec...

  2. 相似度: 0.521
     文檔: 5e2046078b2618a9
     分塊: 5e2046078b2618a9_chunk_1
     內容: Abstract
The dominant sequence transduction models are based on complex recurrent or convolutional n...

  3. 相似度: 0.390
     文檔: 5e2046078b2618a9
     分塊: 5e2046078b2618a9_chunk_6
     內容: 3. Model Architecture
Most competitive neural sequence transduction models have an encoder-decoder s...

--------------------------------------------------
🔎 查詢: 'transformer architecture and self-attention'
🔍 語義搜索找到 3 個結果
找到 3 個相關結果:

  1. 相似度: 0.607
     文檔: 5e2046078b2618a9
     分塊: 5e2046078b2618a9_chunk_4
     內容: In this work, we propose the Transformer, a model architecture eschewing recurrence and instead rely...

  2. 相似度: 0.547
     文檔:

### 測試元資料查詢

In [ ]:
print("📊 測試元資料查詢功能\n")

# 列出所有文檔
all_docs = metadata_manager.list_documents()
print(f"📚 共有 {len(all_docs)} 個文檔:")
for doc in all_docs:
    print(f"  • {doc['title']} ({doc['document_type']})")
    print(f"    字數: {doc['word_count']}, 品質: {doc['quality_score']:.3f}")

print("\n" + "="*60)

# 按類型查詢
academic_papers = metadata_manager.list_documents(document_type="academic_paper")
business_reports = metadata_manager.list_documents(document_type="business_report")

print(f"📖 學術論文: {len(academic_papers)} 篇")
print(f"📈 商業報告: {len(business_reports)} 份")

print("\n" + "="*60)

# 獲取詳細文檔資訊
if processed_documents:
    sample_doc_id = processed_documents[0]['document_id']
    detailed_doc = metadata_manager.get_document(sample_doc_id, include_chunks=True)
    
    print(f"📄 詳細文檔資訊 - {detailed_doc['title']}:")
    print(f"  作者: {', '.join(detailed_doc['authors'])}")
    print(f"  組織: {detailed_doc['organization']}")
    print(f"  類型: {detailed_doc['document_type']}")
    print(f"  分類: {detailed_doc['category']}")
    print(f"  關鍵字: {', '.join(detailed_doc['keywords'][:5])}")
    print(f"  實體: {list(detailed_doc['entities'].keys())}")
    print(f"  分塊數量: {len(detailed_doc.get('chunks', []))}")
    print(f"  自訂元資料: {detailed_doc['custom_metadata']}")

print("\n" + "="*60)

# 生成統計報告
stats = metadata_manager.generate_statistics()
print("📊 系統統計報告:")
for key, value in stats.items():
    print(f"  {key}: {value}")

print("\n" + "="*60)

# 向量索引統計
vector_stats = vector_manager.get_collection_stats()
print("🔍 向量索引統計:")
for key, value in vector_stats.items():
    print(f"  {key}: {value}")

📊 測試元資料查詢功能

📚 共有 4 個文檔:
  • Artificial Intelligence Market Analysis Report 2024 (business_report)
    字數: 195, 品質: 0.000
  • BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding (academic_paper)
    字數: 220, 品質: 0.000
  • Attention Is All You Need (academic_paper)
    字數: 264, 品質: 0.000
  • Attention Is All You Need (academic_paper)
    字數: 264, 品質: 0.000

📖 學術論文: 3 篇
📈 商業報告: 1 份

📄 詳細文檔資訊 - Attention Is All You Need:
  作者: Ashish Vaswani, Noam Shazeer, Niki Parmar
  組織: Google Research
  類型: academic_paper
  分類: deep_learning
  關鍵字: transformer, attention mechanism, neural networks, why self-attention, attention mechanisms
  實體: ['ORG', 'CARDINAL', 'PERSON']
  分塊數量: 7
  自訂元資料: {'venue': 'NIPS 2017', 'citation_count': 50000, 'field': 'Natural Language Processing'}

📊 系統統計報告:
  total_documents: 4
  total_chunks: 20
  avg_chunks_per_doc: 5.0
  document_types: {'academic_paper': 3, 'business_report': 1}
  processing_status: {'completed': 4}
  average_quality_s

### 測試資料血緣追蹤

In [ ]:
print("🔗 測試資料血緣追蹤\n")

# 查看某個文檔的處理歷程
if processed_documents:
    sample_doc_id = processed_documents[0]['document_id']
    lineage_history = metadata_manager.get_lineage_history(sample_doc_id)
    
    print(f"📋 文檔 {sample_doc_id} 的處理歷程:")
    
    for i, record in enumerate(lineage_history, 1):
        print(f"\n{i}. 階段: {record['stage']}")
        print(f"   操作: {record['operation']}")
        print(f"   處理器: {record['processor']}")
        print(f"   時間: {record['timestamp']}")
        print(f"   成功: {'✅' if record['success'] else '❌'}")
        
        if record['input_data']:
            print(f"   輸入: {str(record['input_data'])[:100]}...")
        
        if record['output_data']:
            print(f"   輸出: {str(record['output_data'])[:100]}...")

print("\n" + "="*60)

# 測試相似分塊查找
if processed_documents:
    # 獲取第一個文檔的第一個分塊
    doc_id = processed_documents[0]['document_id']
    doc_detail = metadata_manager.get_document(doc_id, include_chunks=True)
    
    if doc_detail and doc_detail.get('chunks'):
        first_chunk_id = doc_detail['chunks'][0]['chunk_id']
        
        print(f"🔍 查找與分塊 {first_chunk_id} 相似的內容:")
        
        similar_chunks = vector_manager.get_similar_chunks(first_chunk_id, top_k=3)
        
        if similar_chunks:
            for i, chunk in enumerate(similar_chunks, 1):
                print(f"\n{i}. 相似度: {chunk['similarity_score']:.3f}")
                print(f"   來源文檔: {chunk['document_id']}")
                print(f"   內容: {chunk['content'][:150]}...")
        else:
            print("❌ 沒有找到相似的分塊")

print("\n🎉 所有測試完成！")

🔗 測試資料血緣追蹤

📋 文檔 5e2046078b2618a9 的處理歷程:

1. 階段: metadata_creation
   操作: create_document
   處理器: EnterpriseMetadataManager
   時間: 2025-11-11 16:19:38.924093
   成功: ✅
   輸入: {'input': {'title': 'Attention Is All You Need', 'subtitle': None, 'document_type': 'academic_paper'...
   輸出: {'document_id': '5e2046078b2618a9'}...

2. 階段: chunking
   操作: add_chunks
   處理器: EnterpriseMetadataManager
   時間: 2025-11-11 16:19:38.996664
   成功: ✅
   輸入: {'chunk_count': 7}...
   輸出: {'chunk_ids': ['5e2046078b2618a9_chunk_1', '5e2046078b2618a9_chunk_2', '5e2046078b2618a9_chunk_3', '...

🔍 查找與分塊 5e2046078b2618a9_chunk_1 相似的內容:

1. 相似度: 0.729
   來源文檔: 5e2046078b2618a9
   內容: 3. Model Architecture
Most competitive neural sequence transduction models have an encoder-decoder structure. Here, the encoder maps an input sequence...

2. 相似度: 0.691
   來源文檔: 5e2046078b2618a9
   內容: 1. Introduction
Recurrent neural networks, long short-term memory and gated recurrent units in particular, have been firmly establish